In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
# chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [54]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.document_loaders import WebBaseLoader
from langchain.memory import ConversationBufferMemory

import time
OPENAI_KEY = 

with open('script.txt', 'r') as f:
    context = f.read()

customer_template = f"""
You are a customer, you are being sale by a saleman.
You feel uninterested in the product. You feel annoyed while being advertised.
Answer saleman shortly, based on provided context.
Answer must end by an open question.
Answer must be in Vietnamese.

Context information is below.
---------------------
{context}
---------------------
{{chat_history}}
{{message}}
"""

loader = WebBaseLoader("https://eodorshop.com/collections/all")

data = loader.load()
data[0].page_content = data[0].page_content.replace('\n', ' ')

sale_template = f"""
You are a saleman, you are texting with a customer.
You are advertising for a product, the product's information is provided in product context.
Based on product context, answer the customer reasonably an politely.
Answer must be in Vietnamese.
You must call yourself “em" and call customer “anh".

Context information is below.
---------------------
Product context:
{data[0].page_content}
---------------------
{{chat_history}}
{{message}}
"""


customer_prompt = PromptTemplate(template=customer_template, input_variables=['chat_history', 'message'])
sale_prompt = PromptTemplate(template=sale_template, input_variables=['message', 'chat_history'])

memory_customer = ConversationBufferMemory(memory_key="chat_history", ai_prefix="KH", human_prefix="NB")
memory_sale = ConversationBufferMemory(memory_key="chat_history", ai_prefix="NB", human_prefix="KH")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, openai_api_key=OPENAI_KEY)

init_message = 'Xin chào, Anh/Chị [A] phải không ạ?'

is_customer = True
timeout_count = 0

qa_chain_customer = LLMChain(llm=llm, prompt=customer_prompt, memory=memory_customer)
qa_chain_sale = LLMChain(llm=llm, prompt=sale_prompt, memory=memory_sale)


print(f'NB: {init_message}')
while True:
    if is_customer:
        # print('KH')
        init_message = qa_chain_customer.predict(message=init_message)

    else:
        # print('NB') 
        init_message = qa_chain_sale.predict(message=init_message)


    print(init_message)

    is_customer = not (is_customer)
    timeout_count += 1

    if timeout_count == 2: 
        time.sleep(60)
        timeout_count = 0

NB: Xin chào, Anh/Chị [A] phải không ạ?
Ừ là tôi đây, ai đấy?
Em là nhân viên bán hàng của Công ty CP Năng Lượng Đăng Minh. Anh cần tư vấn về sản phẩm nào ạ?
KH: Xin lỗi, nhưng tôi không quan tâm đến sản phẩm của anh. Cảm ơn đã liên hệ.
NB: Xin lỗi anh, nếu anh có bất kỳ câu hỏi hoặc cần tư vấn về sản phẩm nào, em sẽ sẵn lòng giúp đỡ. Cảm ơn anh đã liên hệ.
KH: Không cần, tôi không quan tâm đến sản phẩm của anh. Cảm ơn đã liên hệ. Anh có sản phẩm nào khác không?
NB: Rất tiếc vì sự bất tiện này. Chúng em hiện có nhiều sản phẩm khác mà anh có thể quan tâm. Anh có thể cho em biết những yêu cầu cụ thể của mình để em có thể tư vấn sản phẩm phù hợp hơn được không ạ?


KeyboardInterrupt: 